In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

## Import and curate dataframe

In [5]:
all_dat = pd.read_csv('./curated.csv', index_col=0)

# sift out pwrs
all_dat = all_dat.loc[all_dat['reactor_type'] == 'PWR']  
all_dat = sklearn.utils.shuffle(all_dat)
# only get assemblies with enrichment bigger than 1.5 and bunrup higher than 10,000
all_dat = all_dat.loc[(all_dat['init_enr'] > 1.5) & (all_dat['bu'] > 10000)]

# separate training and testing set
row_num = all_dat.shape[0]
cutoff = int(row_num * 0.6)
train_dat = all_dat.iloc[:cutoff, :]
test_dat = all_dat.iloc[cutoff:, :]


## Categorize Isotopes 

In [6]:
# get all the isotopes, sorted by A
def get_name(f):
    z = ''
    for i in f:
        if i.isalpha():
            z += i
    return z

def get_a(f):
    z = ''
    for i in f:
        if i.isdigit():
            z += i
    return int(z)

iso_list = list(train_dat)[4:]
# set the isotopes to categories:
fp = []
bred_fissile = []
u235 = []
nottru = []
tru = []

for iso in iso_list:
    if iso == 'u-235':
        u235.append(iso)
    elif iso in ['pu-239', 'u-233']:
        bred_fissile.append(iso)
    elif get_a(iso) < 200:
        fp.append(iso)
    elif get_name(iso) in ['np', 'pu', 'am', 'cm']:
        tru.append(iso)
    else:
        nottru.append(iso)

category = {'u235': u235,
            'bred_fissile': bred_fissile,
            'fp': fp,
            'tru': tru,
            'nottru': nottru}

print(category)

{'u235': ['u-235'], 'bred_fissile': ['pu-239', 'u-233'], 'fp': ['zr-95', 'zr-93', 'ru-103', 'ru-106', 'sb-124', 'sb-125', 'nb-95', 'pd-107', 'pm-147', 'i-129', 'h-3', 'kr-85', 'nb-94', 'cs-134', 'cs-135', 'eu-152', 'cs-137', 'eu-154', 'eu-155', 'ag-108m', 'ag-110m', 'c-14', 'ce-144', 'y-90', 'tc-99', 'sm-151', 'se-79', 'sr-90', 'sn-126'], 'tru': ['pu-238', 'np-237', 'pu-240', 'pu-241', 'pu-244', 'pu-242', 'cm-248', 'cm-245', 'cm-244', 'cm-246', 'cm-247', 'am-241', 'am-243', 'cm-243', 'cm-242'], 'nottru': ['ra-226', 'pa-231', 'am-242m', 'cf-251', 'cf-249', 'ac-227', 'u-234', 'u-236', 'u-238', 'th-229', 'th-232', 'th-230', 'u-232']}


# ANN algorithm ( + hyperparameter grid search)
Below is an attempt to fit prediction of all isotopes in one Artificial Neural Network (ANN).
First, the dataset is fit to an array of potential hyperparameters to find the best combination of hyperparameters for best predicting the isotope composition.

In [ ]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [9]:
# grid search to find best hyperparameter

def create_model(optimizer='adam', learn_rate=0.01, momentum=0,
                 activation='relu', neurons=1,
                 hidden_layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=2, activation=activation))
    # consider putting in dropout?
    for i in range(hidden_layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(60, activation='linear'))
    model.compile(loss='mean_absolute_percentage_error', optimizer=optimizer,
                  metrics=['mean_absolute_percentage_error'])
    return model

x = all_dat[['init_enr', 'bu']].as_matrix()
y = all_dat.iloc[:,4:]

model = KerasRegressor(build_fn=create_model, verbose=0)
param_grid = dict(batch_size=[10, 20, 40, 60, 80],
                  epochs=[10, 50, 100, 200],
                  optimizer=['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
                  # learn rate and momentum implementation???
                  activation = ['relu', 'softmax', 'softplus', 'softsign', 'tanh', 'linear'],
                  neurons = [1, 5, 10, 20, 40, 80],
                  hidden_layers = [0, 1, 3, 5, 10]
                  )
result_dict = {}

# for every hyperparameter, run a grid search
for param, param_list in param_grid.items():
    print(param, '\n\n')
    use = {param: param_list}
    grid = GridSearchCV(estimator=model, param_grid=use)
    grid_result = grid.fit(x, y)
    result_dict[param] = (grid_result.best_score_, grid_result.best_params_)
    # summarize results
    print('Best: %f using %s' %(grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    #for mean, stdev. param in zip(means, stds, params):
    #    print('%f (%f) with: %r' %(mean, stdev, param))


/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


batch_size 


Best: -37339.711532 using {'batch_size': 20}
epochs 


Best: -26118.439008 using {'epochs': 100}
optimizer 




/home/teddy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: -64947.839717 using {'optimizer': 'Adamax'}
activation 


Best: -54298.198281 using {'activation': 'softsign'}
neurons 


Best: -72339.409573 using {'neurons': 1}
hidden_layers 


Best: -28876.054232 using {'hidden_layers': 10}


In [19]:
result_dict

{'batch_size': (-37339.71153245486, {'batch_size': 20}),
 'epochs': (-26118.439008011042, {'epochs': 100}),
 'optimizer': (-64947.83971664397, {'optimizer': 'Adamax'}),
 'activation': (-54298.19828140842, {'activation': 'softsign'}),
 'neurons': (-72339.40957284004, {'neurons': 1}),
 'hidden_layers': (-28876.054231980685, {'hidden_layers': 10})}

In [20]:
# use the best:
def get_val(dictionary, key):
    return dictionary[key][1][key]
batch_size = get_val(result_dict, 'batch_size')
epochs = get_val(result_dict, 'epochs')
optimizer = get_val(result_dict, 'optimizer')
activation = get_val(result_dict, 'activation')
neurons = get_val(result_dict, 'neurons')
hidden_layer = get_val(result_dict, 'hidden_layers')
def create_model(optimizer='adam', learn_rate=0.01, momentum=0,
                 activation='relu', neurons=1,
                 hidden_layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=2, activation=activation))
    # consider putting in dropout?
    for i in range(hidden_layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(60, activation='linear'))
    model.compile(loss='mean_absolute_percentage_error', optimizer=optimizer,
                  metrics=['mean_absolute_percentage_error'])
    return model

x = all_dat[['init_enr', 'bu']].as_matrix()
y = all_dat.iloc[:,4:]

model =  create_model(optimizer=optimizer, activation=activation, neurons=neurons,
                     hidden_layers=hidden_layer)
print(model)
model.fit(x, y, epochs=epochs, batch_size=batch_size, validation_split=0.3)
model.evaluate


/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Train on 73185 samples, validate on 31366 samples
Epoch 1/100
73185/73185 [==============================] - 24s 334us/step - loss: 88177.8860 - mean_absolute_percentage_error: 88177.8860 - val_loss: 76297.6464 - val_mean_absolute_percentage_error: 76297.6464
Epoch 2/100
73185/73185 [==============================] - 18s 243us/step - loss: 71776.8624 - mean_absolute_percentage_error: 71776.8624 - val_loss: 66387.5748 - val_mean_absolute_percentage_error: 66387.5748
Epoch 3/100
73185/73185 [==============================] - 17s 227us/step - loss: 71383.8011 - mean_absolute_percentage_error: 71383.8011 - val_loss: 86997.0727 - val_mean_absolute_percentage_error: 86997.0727
Epoch 4/100
73185/73185 [==============================] - 16s 218us/step - loss: 72922.7693 - mean_absolute_percentage_error: 72922.7693 - val_loss: 66281.3289 - val_mean_absolute_percentage_error: 66281.3289
Epoch 5/100
73185/73185 [==============================] - 16s 217us/step - loss: 75174.1389 - mean_absolute_p

73185/73185 [==============================] - 16s 212us/step - loss: 68745.6717 - mean_absolute_percentage_error: 68745.6717 - val_loss: 100072.1471 - val_mean_absolute_percentage_error: 100072.1471
Epoch 40/100
73185/73185 [==============================] - 16s 214us/step - loss: 68765.0778 - mean_absolute_percentage_error: 68765.0778 - val_loss: 71391.0159 - val_mean_absolute_percentage_error: 71391.0159
Epoch 41/100
73185/73185 [==============================] - 16s 212us/step - loss: 69671.2282 - mean_absolute_percentage_error: 69671.2282 - val_loss: 72415.5808 - val_mean_absolute_percentage_error: 72415.5808
Epoch 42/100
73185/73185 [==============================] - 16s 213us/step - loss: 69786.3615 - mean_absolute_percentage_error: 69786.3615 - val_loss: 81661.7223 - val_mean_absolute_percentage_error: 81661.7223
Epoch 43/100
73185/73185 [==============================] - 15s 212us/step - loss: 71513.1146 - mean_absolute_percentage_error: 71513.1146 - val_loss: 70954.3089 - val

Epoch 78/100
73185/73185 [==============================] - 15s 208us/step - loss: 70375.6989 - mean_absolute_percentage_error: 70375.6989 - val_loss: 74180.0553 - val_mean_absolute_percentage_error: 74180.0553
Epoch 79/100
73185/73185 [==============================] - 16s 212us/step - loss: 70814.4295 - mean_absolute_percentage_error: 70814.4295 - val_loss: 85027.3690 - val_mean_absolute_percentage_error: 85027.3690
Epoch 80/100
73185/73185 [==============================] - 15s 211us/step - loss: 70454.8263 - mean_absolute_percentage_error: 70454.8263 - val_loss: 86863.8476 - val_mean_absolute_percentage_error: 86863.8476
Epoch 81/100
73185/73185 [==============================] - 15s 210us/step - loss: 70866.0745 - mean_absolute_percentage_error: 70866.0745 - val_loss: 90682.0947 - val_mean_absolute_percentage_error: 90682.0947
Epoch 82/100
73185/73185 [==============================] - 15s 210us/step - loss: 69912.6575 - mean_absolute_percentage_error: 69912.6575 - val_loss: 52640

<bound method Sequential.evaluate of <keras.models.Sequential object at 0x7f0e9c2dd710>>

## Test the trained model

In [ ]:
testx = x[300]
prediction = model.predict(x)
iso_list = list(y)

In [ ]:
err = []

for indx, val in enumerate(prediction[300]):
    err.append((val-np.array(y.iloc[300])[indx]) / np.array(y.iloc[300])[indx])
    print(iso_list[indx], '\t\t', val, '\t\t', np.array(y.iloc[300])[indx])

print('\n\n\n')
    
for indx, val in enumerate(err):
    print(iso_list[indx], '\t', err[indx])